In [ ]:
!pip install qiskit
!pip install qiskit_aer
!pip install qiskit_machine_learning
!pip install qiskit_algorithms
!pip install matplotlib
!pip install pylatexenc
!pip install scipy
!pip install -U scikit-learn

!git clone https://github.com/IsaVia777/atelier_qml.git

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys 

from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit_machine_learning.optimizers import COBYLA, SPSA
from qiskit.circuit import Parameter
from qiskit.circuit.library import ZZFeatureMap, TwoLocal, ZFeatureMap
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit_machine_learning.circuit.library import QNNCircuit

SEED = 8398

In [ ]:
import sys
sys.path.insert(0, '/content/atelier_qml')

from utils import *

# Lab 4: Entraînement d'un classificateur quantique
**Objectifs**
* Bâtir un circuit paramétré
* Classification du jeu de données Iris

In [ ]:
x_train,y_train,x_test,y_test = get_iris(SEED)
nb_features = 4
nb_classes = 2

# Assemblage d'un classificateur quantique

## Étape 1:  Circuit quantique paramétré

Le circuit quantique paramétré est composé de deux éléments: 

* Un circuit d'encodage des données 
* Un circuit pour apprendre la base de mesure 

In [ ]:
# Circuit d'encodage des données
x_params = [Parameter(f'x{str(i)}') for i in range(nb_features)]    
emb_circuit = angle_embedding(x_params, nb_features)

# Circuit pour apprendre la base de mesure
ansatz = TwoLocal(nb_features, ['rz', 'rx'], 'cx', 'linear', reps=2, parameter_prefix='w')

qc = emb_circuit.compose(ansatz)
qc.draw('mpl')

In [ ]:
qc.decompose().draw('mpl')

## Circuit QNN

In [ ]:
# Initialisation aléatoire des paramètres
np.random.seed(SEED)
initial_weights = np.random.rand(ansatz.num_parameters)

In [ ]:
# Fonction d'interprétation de la mesure: la parité compte le nombre
# de "1" dans la chaîne de bits, x,
# et retourne 0 si ce nombre est pair, 1 si impair
def parity(x):
    return '{:b}'.format(x).count('1') % 2

In [ ]:
# Initialisation de l'optimiseur
num_iter = 20
optimizer = COBYLA(maxiter=num_iter)

sampler_qnn = SamplerQNN(circuit=qc,  
                         input_params=emb_circuit.parameters,  ## Mettre la liste des paramètres ici!
                         weight_params=ansatz.parameters, 
                         interpret=parity, 
                         output_shape=nb_classes)

In [ ]:
# Le vecteur des probabilités d'appartenir à la classe 0 ou 1 pour un point donné
# et une valeur donnée des paramètres s'obtient avec la méthode `CircuitQNN.forward()`
probs = sampler_qnn.forward(x_train[0], initial_weights)
print(f">\n> Probabilité d'appartenir à la classe 0: {probs[0][0]*100:.1f}%\n> Probabilité d'appartenir à la classe 1: {probs[0][1]*100:.1f}%\n>")

## Entraînement du classificateur avec le `NeuralNetworkClassifier`

In [ ]:
# Instanciation de la classe utilisée pour entraîner le classificateur quantique
circuit_classifier = NeuralNetworkClassifier(neural_network=sampler_qnn,
                                             optimizer=optimizer,
                                             initial_point=initial_weights)
# Entraînement du model
circuit_classifier.fit(x_train, y_train)

In [ ]:
# Calcul de la précision du classificateur sur les ensembles d'entraînement et de test
train_acc = circuit_classifier.score(x_train, y_train)
test_acc = circuit_classifier.score(x_test, y_test)

print(f">\n> Précision sur l'ensemble d'entraînement: {train_acc}\n> Précision sur l'ensemble de test: {test_acc}\n>")

## Exercice 4

Est-ce qu'on peut faire mieux??<br>
Essayons avec un circuit d'encodage des données plus riche que l'encodage par angle, par exemple `ZZFeatureMap`.

In [ ]:
## Votre code ici

feature_map = None
##

# On utilise le même circuit pour apprendre la base de mesure 
ansatz = TwoLocal(nb_features, ['rz', 'rx'], 'cx', 'linear', reps=2, parameter_prefix='w')

qc = feature_map.compose(ansatz)
qc.draw('mpl')

In [ ]:
## Votre code ici

circuit_qnn = SamplerQNN(circuit=qc,  
                         input_params=None,  ## Mettre la liste des paramètres ici!
                         weight_params=ansatz.parameters, 
                         interpret=parity, 
                         output_shape=nb_classes,  
                         quantum_instance=qinst)

In [ ]:
# Instanciation de la classe utilisée pour entraîner le classificateur quantique
circuit_classifier = NeuralNetworkClassifier(neural_network=circuit_qnn,
                                             optimizer=optimizer,
                                             initial_point=initial_weights)

# Entraînement du model
circuit_classifier.fit(x_train, y_train)

In [ ]:
# Calcul de la précision du classificateur sur les ensembles d'entraînement et de test
train_acc = circuit_classifier.score(x_train, y_train)
test_acc = circuit_classifier.score(x_test, y_test)

print(f">\n> Précision sur l'ensemble d'entraînement: {train_acc}\n> Précision sur l'ensemble de test: {test_acc}\n>")

Pour explorer, vous pouvez faire varier les éléments suivants:
* circuit d'encodage
* optimiseur (nombre d'itération ou encore un autre optimiser, par exemple SPSA)
* fonction d'interprétation de la mesure